### L'objectif de ce notebook est de comparer les notes données à la culture et la présence ou non de lieux culturels par arrondissement.

Installation et importation des librairies nécessaires :

In [2]:
!pip install openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Commençons par nous intéresser aux notes attribuées pour la culture de chaque arrondissement :

In [7]:
# Création d'un dataframe NotesCulture contenant les données dont nous avons besoin
Notes = pd.read_excel('data/Notes_arrondissements.xlsx', decimal=',', index_col=0)
NotesCulture = Notes['Culture']
NotesCulture

Paris 1er    8.58
Paris 2e     8.00
Paris 3e     9.33
Paris 4e     7.46
Paris 5e     8.50
Paris 6e     8.40
Paris 7e     7.24
Paris 8e     7.44
Paris 9e     8.50
Paris 10e    6.18
Paris 11e    7.01
Paris 12e    6.90
Paris 13e    6.76
Paris 14e    6.54
Paris 15e    6.72
Paris 16e    6.66
Paris 17e    6.65
Paris 18e    5.89
Paris 19e    7.01
Paris 20e    6.59
Name: Culture, dtype: float64

In [8]:
# Affichage des 4 meilleurs et des 4 pires arrondissement pour leur culture, d'après les notes des habitants
print('Les 4 arrondissements ayant les meilleures notes pour leur culture sont :')
display(NotesCulture.sort_values(ascending=False)[:4])

print('Les 4 arrondissements ayant les pires notes pour leur culture sont :')
display(NotesCulture.sort_values()[:4])

Les 4 arrondissements ayant les meilleures notes pour leur culture sont :


Paris 3e     9.33
Paris 1er    8.58
Paris 9e     8.50
Paris 5e     8.50
Name: Culture, dtype: float64

Les 4 arrondissements ayant les pires notes pour leur culture sont :


Paris 18e    5.89
Paris 10e    6.18
Paris 14e    6.54
Paris 20e    6.59
Name: Culture, dtype: float64

En ce qui concerne les lieux culturels, les Parisiens semblent donc avoir une préférence pour les arrondissements 3, 1, 9 et 5.
Le troisième arrondissement est largement en tête avec une note de 9.33.
Le premier, neuvième et cinquième arrondissement forment un deuxieme groupe en tête avec une note autour de 8.5.

Les arrondissements 18, 10, 14 et 20 obtiennent eux les pires notes : leurs notes sont réparties entre 5.89 pour le 18ème et 6.59.

Voyons quelques statistiques sur les notes données pour la culture :

In [107]:
NotesCulture.describe()

count    20.000000
mean      7.318000
std       0.931725
min       5.890000
25%       6.657500
50%       7.010000
75%       8.100000
max       9.330000
Name: Culture, dtype: float64

Ces premières statistiques descriptives nous apprennent que le top 4 des arrondissement se situent au dessus de 75% des notes, ce qui est normal mathématiquement.
La note moyenne obtenue pour la culture est de 7.32, ce qui est haut. L'écart-type est de 0.93 ce qui est assez bas. Les notes sont
plutôt concentrées autour de 7. 

Si on compare donc les pires et meilleures notes, c'est surtout le 3ème arrondissement qui se démarque avec son 9.33. La pire note, 5.89, n'est pas autant éloignée du groupe que cela.

Maintenant, nous allons importer la base de données recensant tous les lieux culturels parisiens. Nous pourrons alors voir s'il y a une corrélation entre le nombre de lieux culturels et la satisfaction des habitants parisiens.

### Import de la base de données recensant les divers lieux culturels de la capitale

In [12]:
# La base de données vient de data.gouv.fr
LieuxCulturels = pd.read_csv('data/lieux_culturels.csv')
display(LieuxCulturels)

,NOM_LIEU,DEP,COMMUNE,CODE_INSEE,CODE_POSTAL,RAYONNEMENT,GPE_800M,GPE_STATION,GPE_LIGNE,CARTO_BIB,...,C_SUF3,C_DESI,C_LIAISON,L_VOIE,N_SQ_EP,C_NIV1,C_NIV2,C_NIV3,COORD_X,COORD_Y
0,Acadmie de Danse Barbot Perrot - Danse,75.0,Paris 17e Arrondissement,75117.0,75017.0,3.0,Oui,Pont Cardinet,GPE - L14,NaN,...,NaN,RUE,DE,TOCQUEVILLE,753000955.0,13.0,132.0,13211.0,6.496656e+09,6.864963e+09
1,Acadmie de Guitare,75.0,Paris 13e Arrondissement,75113.0,75013.0,3.0,Oui,Maison Blanche - Paris XIIIe,GPE - L14,NaN,...,NaN,PAS,DU,MOULINET,753000741.0,13.0,132.0,13211.0,6.525824e+09,6.858678e+09
2,Association Cours Hippolyte,75.0,Paris 17e Arrondissement,75117.0,75017.0,3.0,Oui,Pont Cardinet,GPE - L14,NaN,...,NaN,RUE,NaN,CARDINET,753000238.0,13.0,132.0,13211.0,6.494944e+09,6.865273e+09
3,Atelier d'Artistes,75.0,Paris 17e Arrondissement,75117.0,75017.0,3.0,Oui,Pont Cardinet,GPE - L14,NaN,...,NaN,RUE,DES,DAMES,753000100.0,13.0,134.0,13413.0,6.498860e+09,6.864952e+09
4,Atelier de Recherche Chorgraphique - Danse,75.0,Paris 13e Arrondissement,75113.0,75013.0,3.0,Oui,Maison Blanche - Paris XIIIe,GPE - L14,NaN,...,NaN,AV,DE,CHOISY,753000625.0,13.0,132.0,13211.0,6.528911e+09,6.858847e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,Ya d'la Voix!,75.0,Paris 11e Arrondissement,75111.0,75011.0,3.0,Non,NaN,NaN,NaN,...,NaN,RUE,DU,FAUBOURG SAINT-ANTOINE,753000231.0,13.0,132.0,13211.0,6.545428e+09,6.861345e+09
1103,Yamaha Music School,75.0,Paris 12e Arrondissement,75112.0,75012.0,3.0,Non,NaN,NaN,NaN,...,NaN,RUE,DE,BERCY,753000573.0,13.0,132.0,13211.0,6.545667e+09,6.860013e+09
1104,Yvon Lambert,75.0,Paris 3e Arrondissement,75103.0,75003.0,3.0,Non,NaN,NaN,NaN,...,NaN,RUE,NaN,VIEILLE DU TEMPLE,753000707.0,13.0,133.0,13321.0,6.532523e+09,6.862514e+09
1105,Archives Nationales Site Paris,75.0,Paris 3e Arrondissement,75103.0,75003.0,1.0,Non,NaN,NaN,NaN,...,NaN,RUE,DES,FRANCS BOURGEOIS,751009138.0,13.0,131.0,13121.0,6.528442e+09,6.862406e+08


### Préparation de cette base de données pour notre étude

In [13]:
# Suppression des '.0' dans la liste des codes postaux par soucis d'esthétisme :
LieuxCulturels.CODE_POSTAL = LieuxCulturels.CODE_POSTAL.astype(str)
LieuxCulturels.CODE_POSTAL = LieuxCulturels.CODE_POSTAL.str.replace('.0','')

In [14]:
# Suppression des lignes n'étant pas associées à un arrondissement :
LieuxCulturels.drop(LieuxCulturels[LieuxCulturels.CODE_POSTAL == 'nan'].index, inplace=True)

In [15]:
# Transformation de la base pour qu'elle recense uniquement le nombre de lieux culturels par arrondissement.
NbLieux = LieuxCulturels.CODE_POSTAL.value_counts()

In [16]:
# Affichage des 4 arrondissement ayant le plus (et le moins) de lieux culturels
print('Les 4 arrondissements ayant le plus de lieux culturels sont :')
display(NbLieux.sort_values(ascending=False)[:4])

print('Les 4 arrondissements ayant le moins de lieux culturels sont :')
display(NbLieux.sort_values()[:4])

Les 4 arrondissements ayant le plus de lieux culturels sont :


CODE_POSTAL
75011    90
75019    72
75018    71
75006    69
Name: count, dtype: int64

Les 4 arrondissements ayant le moins de lieux culturels sont :


CODE_POSTAL
75002    21
75007    29
75001    31
75003    47
Name: count, dtype: int64

Aucun des 4 arrondissements ayant les meilleures note pour leur culture ne se trouvent dans la liste des 4 arrondissements ayant le plus de lieux culturels. Il semble donc venir de cette analyse que la qualité prime sur la quantité de lieux culturels pour la satisfaction des habitants.

Les 4 arrondissements ayant les pires notes ne sont absolument pas ceux ayant le moins d'institutions culturelles non plus. La conclusion semble alors se confirmer. Cela va même encore plus loin : le 3ème arrondissement, qui est de loin le préféré des Parisiens sur le critère de la culture, est le 4ème arrondissement ayant le moins de lieux culturels !


##### Piste d'amélioration de l'analyse :

Plutot que de s'intéresser au nombre brut de lieux culturels par arrondissement; nous allons prendre la densité de lieux culturels comme variable d'intérêt. Nous allons donc inclure la superficie des arrondissements dans l'analyse pour voir si les résultats
obtenus sont davantage cohérents.

In [30]:
# Création d'une base de donnée associant pour chaque arrondissement sa superficie.
# Import des données géographiques des arrondissements parisiens depuis la librairie cartiflette

try:
    !pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
    !pip install git+https://github.com/inseefrlab/cartogether
    import cartiflette.s3 as s3

    Arrondissements = s3.download_vectorfile_url_all(
        crs = 4326,
        values = ["75"],
        borders="COMMUNE_ARRONDISSEMENT",
        vectorfile_format="topojson",
        filter_by="DEPARTEMENT",
        source="EXPRESS-COG-CARTO-TERRITOIRE",
        year=2022)

    # Stockage des superficies dans la Pandas Series superficies

    Arrondissements = Arrondissements.sort_values('INSEE_COG').reset_index()
    superficies = Arrondissements.area
    superficies

except Exception as e:
    print('La librairie cartiflette semble avoir un problème :')
    print(e)
    print('La Pandas Series superficie est donc créée manuellement')
    superficies = pd.Series([0.000224,0.000122,0.000143,0.000196,0.000312,0.000264,0.000501,0.000476,0.000267,0.000354,0.000447,0.002007,0.000874,0.000688,0.001038,0.002011,0.000695,0.000735,0.000832,0.000736])
    superficies

  Cloning https://github.com/inseefrlab/cartogether to /tmp/pip-req-build-slo801jg
  Running command git clone --filter=blob:none --quiet https://github.com/inseefrlab/cartogether /tmp/pip-req-build-slo801jg
  Resolved https://github.com/inseefrlab/cartogether to commit 7a358b27121bfcd1060c05fd0ca8b3a2d2747dad
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
La librairie cartiflette semble avoir un problème :
module 'cartiflette.s3' has no attribute 'download_vectorfile_url_all'
La Pandas Series superficie est donc créée manuellement


Notes : nous ne nous interessons pas à l'unité de mesure des superficies, le rapport entre les unes et les autres nous suffira pour comparer les densités.

In [31]:
NbLieux.to_frame().sort_values('CODE_POSTAL').reset_index()

,CODE_POSTAL,count
0,75001,31
1,75002,21
2,75003,47
3,75004,49
4,75005,51
5,75006,69
6,75007,29
7,75008,60
8,75009,63
9,75010,48


In [32]:
# Ajout des données de superficie au dataframe contenant le nombre de lieux culturels par arrondissement.

NbLieux = NbLieux.to_frame().sort_values('CODE_POSTAL').reset_index()
NbLieux['superficie'] = superficies.values
NbLieux = NbLieux.rename(columns={'count':'nb_lieux'})

In [33]:
# Création d'une colonne contenant les densités des lieux culturels 
NbLieux.nb_lieux = NbLieux.nb_lieux.astype('float64')
NbLieux['densite_culture'] = NbLieux.nb_lieux / NbLieux.superficie

In [34]:
display(NbLieux.sort_values('densite_culture', ascending=False))

,CODE_POSTAL,nb_lieux,superficie,densite_culture
2,75003,47.0,0.000143,328671.328671
5,75006,69.0,0.000264,261363.636364
3,75004,49.0,0.000196,250000.000000
8,75009,63.0,0.000267,235955.056180
10,75011,90.0,0.000447,201342.281879
1,75002,21.0,0.000122,172131.147541
4,75005,51.0,0.000312,163461.538462
0,75001,31.0,0.000224,138392.857143
9,75010,48.0,0.000354,135593.220339
7,75008,60.0,0.000476,126050.420168


Le résultat est convaincant : le 3ème arrondissement, qui dispose de la meilleure note - et de loin - pour sa culture, est passé à la première place sur la classement des arrondissements disposant du plus de lieux culturels au mètre carré. Rappelons qu'il n'était même pas dans le top 4 des arrondissements disposant du plus de lieux culturels en nombre brut. 

Alors qu'elle n'apparaissait pas avant de considérer cette approche, nous pouvons maintenant supposer que la richesse du nombre de lieux culturels dans le troisième arrondissement explique en partie la note moyenne donnée par les habitants.

Les arrondissements 12 et 16 se retrouvent en bas de liste en terme de densité de lieux culturels : cela s'explique par le fait que la superficie prend en compte les bois de Boulogne et de Vincennes qui gonflent de façon énorme la valeur de leur superficie. 